In [ ]:
import csv
import pandas as pd
import numpy as np
import re

health = pd.read_csv("health-data.csv")
health.head()

,Date,Active Calories (kcal),Blood Glucose (mg/dL),Blood Oxygen (%),Blood Pressure (mmHg),Body Fat (%),Body Mass Index,Body Temperature (degF),Caffeine (mg),Calcium (mg),...,Walking + Running (mi),Walking Asymmetry (%),Walking Heart Rate (bpm),Walking Speed (mi/hr),Water (US oz),Weight (lb),Wheelchair Distance (mi),Workout Type,DURATION,Zinc (mg)
0,9/19/23,847,-,-,-,-,-,-,-,-,...,7.16,3.01,106,1.07-3.53,-,-,-,NaN,NaN,-
1,9/20/23,990,-,-,-,-,-,-,-,-,...,8.61,5.62,132,1.43-3.58,-,-,-,Walking,31m,-
2,9/21/23,"1,042",-,-,-,-,-,-,-,-,...,9.40,14.34,-,1.12-3.91,-,-,-,NaN,NaN,-
3,9/22/23,595,-,-,-,-,-,-,-,-,...,7.85,7.02,112.5,1.32-3.51,-,-,-,NaN,NaN,-
4,9/23/23,829,-,-,-,-,-,-,-,-,...,6.49,19.06,-,1.28-3.06,-,-,-,NaN,NaN,-


In [ ]:
health.columns = health.columns.str.lower().str.replace(" ", "_")
print(health.columns)

Index(['date', 'active_calories_(kcal)', 'blood_glucose_(mg/dl)',
       'blood_oxygen_(%)', 'blood_pressure_(mmhg)', 'body_fat_(%)',
       'body_mass_index', 'body_temperature_(degf)', 'caffeine_(mg)',
       'calcium_(mg)', 'calories_(kcal)', 'carbohydrates_(g)',
       'cardio_fitness_(ml/min·kg)', 'chloride_(mg)', 'cholesterol_(mg)',
       'chromium_(mcg)', 'copper_(mg)', 'cycling_distance_(mi)',
       'dietary_sugar_(g)', 'double_support_time_(%)', 'exercise_minutes',
       'fiber_(g)', 'flights_climbed_(floors)', 'folate_(mcg)',
       'forced_expiratory_volume,_1_sec_(l)', 'handwashing',
       'heart_rate_(bpm)', 'heart_rate_variability_(ms)', 'height_(cm)',
       'insulin_delivery_(u)', 'iodine_(mcg)', 'iron_(mg)',
       'lean_body_mass_(lb)', 'magnesium_(mcg)', 'manganese_(mg)',
       'mindful_minutes', 'molybdenum_(mcg)', 'monounsaturated_fat_(g)',
       'niacin_(mg)', 'nike_fuel', 'pantothenic_acid_(mg)',
       'peak_expiratory_flow_rate_(l/min)', 'phosphorus_(mg)'

In [ ]:
health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906 entries, 0 to 905
Data columns (total 84 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   date                                 906 non-null    object 
 1   active_calories_(kcal)               746 non-null    object 
 2   blood_glucose_(mg/dl)                746 non-null    object 
 3   blood_oxygen_(%)                     746 non-null    object 
 4   blood_pressure_(mmhg)                746 non-null    object 
 5   body_fat_(%)                         746 non-null    object 
 6   body_mass_index                      746 non-null    object 
 7   body_temperature_(degf)              746 non-null    object 
 8   caffeine_(mg)                        746 non-null    object 
 9   calcium_(mg)                         746 non-null    object 
 10  calories_(kcal)                      746 non-null    object 
 11  carbohydrates_(g)               

In [ ]:
# parsing timestamp from col 0
health["timestamp"] = pd.to_datetime(
    health["date"],
    format="%m-%d-%Y",
    errors="coerce"
)

In [ ]:
health.head()

,date,active_calories_(kcal),blood_glucose_(mg/dl),blood_oxygen_(%),blood_pressure_(mmhg),body_fat_(%),body_mass_index,body_temperature_(degf),caffeine_(mg),calcium_(mg),...,walking_asymmetry_(%),walking_heart_rate_(bpm),walking_speed_(mi/hr),water_(us_oz),weight_(lb),wheelchair_distance_(mi),workout_type,_duration,zinc_(mg),timestamp
0,9/19/23,847,-,-,-,-,-,-,-,-,...,3.01,106,1.07-3.53,-,-,-,NaN,NaN,-,NaT
1,9/20/23,990,-,-,-,-,-,-,-,-,...,5.62,132,1.43-3.58,-,-,-,Walking,31m,-,NaT
2,9/21/23,"1,042",-,-,-,-,-,-,-,-,...,14.34,-,1.12-3.91,-,-,-,NaN,NaN,-,NaT
3,9/22/23,595,-,-,-,-,-,-,-,-,...,7.02,112.5,1.32-3.51,-,-,-,NaN,NaN,-,NaT
4,9/23/23,829,-,-,-,-,-,-,-,-,...,19.06,-,1.28-3.06,-,-,-,NaN,NaN,-,NaT


In [ ]:
keep_cols = [
    "date",
    "active_calories_(kcal)",
    "exercise_minutes",
    "flights_climbed_(floors)",
    "heart_rate_(bpm)",
    "resting_calories_(kcal)",
    "resting_heart_rate_(bpm)",
    "six-minute_walk_(m)",
    "stair_speed:_down_(ft/s)",
    "stair_speed:_up_(ft/s)",
    "step_length_(in)",
    "steps_(steps)",
    "walking_+_running_(mi)",
    "walking_heart_rate_(bpm)",
    "walking_speed_(mi/hr)",
    "heart_rate_variability_(ms)",
    "cardio_fitness_(ml/min·kg)"
]

health_filtered = health[keep_cols]
health_filtered.to_csv("health-data-filtered.csv", index=False)

print("Original shape: " + str(health.shape))
print("Filtered shape: " + str(health_filtered.shape))

Original shape: (906, 85)
Filtered shape: (906, 17)


In [ ]:
health_filtered.head()

,date,active_calories_(kcal),exercise_minutes,flights_climbed_(floors),heart_rate_(bpm),resting_calories_(kcal),resting_heart_rate_(bpm),six-minute_walk_(m),stair_speed:_down_(ft/s),stair_speed:_up_(ft/s),step_length_(in),steps_(steps),walking_+_running_(mi),walking_heart_rate_(bpm),walking_speed_(mi/hr),heart_rate_variability_(ms),cardio_fitness_(ml/min·kg)
0,9/19/23,847,67,6,56-175,"1,837",-,-,0.73-1.15,-,12.6-31.5,"15,420",7.16,106,1.07-3.53,19.04-80.8,-
1,9/20/23,990,112,21,65-166,"1,845",68,-,0.93-1.97,0.88-0.91,18.11-35.04,"18,577",8.61,132,1.43-3.58,16.62-52.36,25.09
2,9/21/23,"1,042",87,17,53-131,"1,880",59,-,1.06-1.16,0.71-1.11,15.35-40.55,"20,775",9.40,-,1.12-3.91,30.64-118.02,-
3,9/22/23,595,38,15,53-147,"1,723",57,-,0.77,0.81-1.17,15.35-31.89,"17,906",7.85,112.5,1.32-3.51,38.21-121.43,-
4,9/23/23,829,29,3,54-108,"1,890",59,-,1-1.14,-,17.32-31.1,"14,350",6.49,-,1.28-3.06,39.84-166.04,-


In [ ]:
print(health_filtered.dtypes)


date                            object
active_calories_(kcal)          object
exercise_minutes                object
flights_climbed_(floors)        object
heart_rate_(bpm)                object
resting_calories_(kcal)         object
resting_heart_rate_(bpm)        object
six-minute_walk_(m)             object
stair_speed:_down_(ft/s)        object
stair_speed:_up_(ft/s)          object
step_length_(in)                object
steps_(steps)                   object
walking_+_running_(mi)         float64
walking_heart_rate_(bpm)        object
walking_speed_(mi/hr)           object
heart_rate_variability_(ms)     object
cardio_fitness_(ml/min·kg)      object
dtype: object


In [ ]:
def expand_ranges(health_filtered: pd.DataFrame):
    health_filtered["date"] = pd.to_datetime(health_filtered["date"], format="%m/%d/%y", errors="coerce")

    range_re = re.compile(r"^\s*([-+]?\d+(?:\.\d+)?)\s*-\s*([-+]?\d+(?:\.\d+)?)\s*$")

    def low_high(s: str):
        if pd.isna(s):
            return (np.nan, np.nan)
        s = s.replace(",", " ").replace("\u00A0", " ").strip()
        if s in {"", "-"}:
            return (np.nan, np.nan)

        m = range_re.match(s)
        if m:
            a = float(m.group(1))
            b = float(m.group(2))
            return (a, b)

        cleaned = re.sub(r"[^0-9.\-]", "", s)
        if cleaned in {"", "-", ".", "-.", ".-"}:
            return (np.nan, np.nan)

        try:
            x = float(cleaned)
            return (x,x)
        except ValueError:
            return (np.nan, np.nan)

    skip_cols = {"date"}
    obj_cols = [c for c in health_filtered.columns if health_filtered[c].dtype == "object" and c.lower() not in skip_cols]

    for col in obj_cols:
        lows, highs = zip(*(low_high(v) for v in health_filtered[col].tolist()))
        low_col, high_col, mid_col = f"{col}_low", f"{col}_high", f"{col}_mid"
        health_filtered[low_col] = pd.to_numeric(lows, errors="coerce")
        health_filtered[high_col] = pd.to_numeric(highs, errors="coerce")
        health_filtered[mid_col] = (health_filtered[low_col] + health_filtered[high_col]) / 2.0

        return health_filtered

In [ ]:
health_filtered = expand_ranges(health_filtered)
# FIGURE OUT HOW TO CONVERT RANGES TO DIFF COLS LOL

3 new columns
['active_calories_(kcal)_low', 'active_calories_(kcal)_high', 'active_calories_(kcal)_mid']


/var/folders/pv/fhr4_78j6ys6_8wkxsh5w88r0000gn/T/ipykernel_2183/801940392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  health_filtered["date"] = pd.to_datetime(health_filtered["date"], format="%m/%d/%y", errors="coerce")
/var/folders/pv/fhr4_78j6ys6_8wkxsh5w88r0000gn/T/ipykernel_2183/801940392.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  health_filtered[low_col] = pd.to_numeric(lows, errors="coerce")
/var/folders/pv/fhr4_78j6ys6_8wkxsh5w88r0000gn/T/ipykernel_2183/801940392.py:36: SettingWithC

In [ ]:
num = health_filtered.select_dtypes(include="number")
summary = num.agg(["min", "max", "mean", "std"])

print(summary)

      walking_+_running_(mi)  active_calories_(kcal)_low  \
min                 0.390000                    0.000000   
max                16.720000                 1627.000000   
mean                4.952654                  583.800842   
std                 2.378258                  256.347107   

      active_calories_(kcal)_high  active_calories_(kcal)_mid  
min                      0.000000                    0.000000  
max                   1627.000000                 1627.000000  
mean                   583.800842                  583.800842  
std                    256.347107                  256.347107  
